This notebook read the output of the model to extract only the relation that have a score higher than a predefine threshold. As the output of the model is a simple `json` this can be done in Scala and the detected relation stored in HBase.

In [1]:
import json
from itertools import groupby

In [2]:
path_to_result = '../test_result/nyt_pcnn_att_pred.json'
path_rel2id = '../data/nyt/rel2id.json'

In [3]:
with open(path_to_result, 'r') as f:
    result = json.load(f)

In [5]:
with open(path_rel2id, 'r') as f:
    rel2id = json.load(f)

In [6]:
id2rel = {v:str(k) for k, v in rel2id.items()}

In [7]:
threshold = 0.5
relations = []
for i, grp in groupby(result, lambda x: x['entpair']):
    for row in grp:
        if row['score'] >= threshold:
            assert row['relation'] != 0
            ent1 = row['entpair'].split('#')[0]
            ent2 = row['entpair'].split('#')[1]
            relations.append((ent1, id2rel[row['relation']], ent2, row['score']))

In [8]:
id2ent = json.load(open('id2ent.json', 'r'))

In [9]:
import pandas as pd
df_res = pd.DataFrame(relations, columns=['ent 1', 'rel', 'ent 2', 'score'])
df_res['ent 1 label'] = df_res['ent 1'].apply(lambda x: id2ent.get(x, 'UNK'))
df_res['ent 2 label'] = df_res['ent 2'].apply(lambda x: id2ent.get(x, 'UNK'))

In [10]:
df_res.sort_values('ent 1').set_index(['ent 1 label', 'rel'])[['ent 2 label', 'score']]

,,ent 2 label,score
ent 1 label,rel,,
New York,/location/location/contains,Bastien,0.011510
Bastien,/location/location/contains,New York,0.011510
2019,/business/person/company,MIT,0.010135
Peter,/business/person/company,MIT,0.010135
